### Matrix to bytes

In [1]:
def bytes2matrix(text):
    """ Converts a 16-byte array into a 4x4 matrix.  """
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array.  """
    return bytes([byte for row in matrix for byte in row])

matrix = [
    [99, 114, 121, 112],
    [116, 111, 123, 105],
    [110, 109, 97, 116],
    [114, 105, 120, 125],
]

print(matrix2bytes(matrix))


b'crypto{inmatrix}'


### Round Keys

In [3]:
state = [
    [206, 243, 61, 34],
    [171, 11, 93, 31],
    [16, 200, 91, 108],
    [150, 3, 194, 51],
]

round_key = [
    [173, 129, 68, 82],
    [223, 100, 38, 109],
    [32, 189, 53, 8],
    [253, 48, 187, 78],
]


def add_round_key(s, k):
    return [[s[i][j] ^ k[i][j] for j in range(4)] for i in range(4)]


round_key_val = add_round_key(state, round_key)
print(round_key_val)

val = matrix2bytes(round_key_val)
print(val)

[[99, 114, 121, 112], [116, 111, 123, 114], [48, 117, 110, 100], [107, 51, 121, 125]]
b'crypto{r0undk3y}'


### Confusion Substitution

### Diffusion Permutation

In [5]:
def shift_rows(s):
    s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]


def inv_shift_rows(s):
    s[1][1], s[2][1], s[3][1], s[0][1] = s[0][1], s[1][1], s[2][1], s[3][1]
    s[2][2], s[3][2], s[0][2], s[1][2] = s[0][2], s[1][2], s[2][2], s[3][2]
    s[3][3], s[0][3], s[1][3], s[2][3] = s[0][3], s[1][3], s[2][3], s[3][3]


# learned from http://cs.ucsb.edu/~koc/cs178/projects/JT/aes.c
xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)


def mix_single_column(a):
    # see Sec 4.1.2 in The Design of Rijndael
    t = a[0] ^ a[1] ^ a[2] ^ a[3]
    u = a[0]
    a[0] ^= t ^ xtime(a[0] ^ a[1])
    a[1] ^= t ^ xtime(a[1] ^ a[2])
    a[2] ^= t ^ xtime(a[2] ^ a[3])
    a[3] ^= t ^ xtime(a[3] ^ u)


def mix_columns(s):
    for i in range(4):
        mix_single_column(s[i])


def inv_mix_columns(s):
    # see Sec 4.1.3 in The Design of Rijndael
    for i in range(4):
        u = xtime(xtime(s[i][0] ^ s[i][2]))
        v = xtime(xtime(s[i][1] ^ s[i][3]))
        s[i][0] ^= u
        s[i][1] ^= v
        s[i][2] ^= u
        s[i][3] ^= v

    mix_columns(s)


state = [
    [108, 106, 71, 86],
    [96, 62, 38, 72],
    [42, 184, 92, 209],
    [94, 79, 8, 54],
]

inv_mix_columns(state)
inv_shift_rows(state)
print(bytes(sum(state, [])))

b'crypto{d1ffUs3R}'


### Bringing It All Together

In [6]:
N_ROUNDS = 10

key        = b'\xc3,\\\xa6\xb5\x80^\x0c\xdb\x8d\xa5z*\xb6\xfe\\'
ciphertext = b'\xd1O\x14j\xa4+O\xb6\xa1\xc4\x08B)\x8f\x12\xdd'

s_box = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

inv_s_box = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

def expand_key(master_key):
    """
    Expands and returns a list of key matrices for the given master_key.
    """

    # Round constants https://en.wikipedia.org/wiki/AES_key_schedule#Round_constants
    r_con = (
        0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
        0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
        0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
        0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
    )

    # Initialize round keys with raw key material.
    key_columns = bytes2matrix(master_key)
    iteration_size = len(master_key) // 4

    # Each iteration has exactly as many columns as the key material.
    i = 1
    while len(key_columns) < (N_ROUNDS + 1) * 4:
        # Copy previous word.
        word = list(key_columns[-1])

        # Perform schedule_core once every "row".
        if len(key_columns) % iteration_size == 0:
            # Circular shift.
            word.append(word.pop(0))
            # Map to S-BOX.
            word = [s_box[b] for b in word]
            # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
            word[0] ^= r_con[i]
            i += 1
        elif len(master_key) == 32 and len(key_columns) % iteration_size == 4:
            # Run word through S-box in the fourth iteration when using a
            # 256-bit key.
            word = [s_box[b] for b in word]

        # XOR with equivalent word from previous iteration.
        word = bytes(i^j for i, j in zip(word, key_columns[-iteration_size]))
        key_columns.append(word)

    # Group key words in 4x4 byte matrices.
    return [key_columns[4*i : 4*(i+1)] for i in range(len(key_columns) // 4)]

def bytes2matrix(text):
    """ Converts a 16-byte array into a 4x4 matrix.  """
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array.  """
    return b''.join(bytes([a]) for row in matrix for a in row)

def add_round_key(s, k):
    return [[s[i][j]^k[i][j] for j in range(4)] for i in range(4)]

def inv_shift_rows(s):
    s[1][1], s[2][1], s[3][1], s[0][1] = s[0][1], s[1][1], s[2][1], s[3][1]
    s[2][2], s[3][2], s[0][2], s[1][2] = s[0][2], s[1][2], s[2][2], s[3][2]
    s[3][3], s[0][3], s[1][3], s[2][3] = s[0][3], s[1][3], s[2][3], s[3][3]

# learned from http://cs.ucsb.edu/~koc/cs178/projects/JT/aes.c
xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

def mix_single_column(a):
    # see Sec 4.1.2 in The Design of Rijndael
    t = a[0] ^ a[1] ^ a[2] ^ a[3]
    u = a[0]
    a[0] ^= t ^ xtime(a[0] ^ a[1])
    a[1] ^= t ^ xtime(a[1] ^ a[2])
    a[2] ^= t ^ xtime(a[2] ^ a[3])
    a[3] ^= t ^ xtime(a[3] ^ u)

def mix_columns(s):
    for i in range(4):
        mix_single_column(s[i])

def inv_mix_columns(s):
    # see Sec 4.1.3 in The Design of Rijndael
    for i in range(4):
        u = xtime(xtime(s[i][0] ^ s[i][2]))
        v = xtime(xtime(s[i][1] ^ s[i][3]))
        s[i][0] ^= u
        s[i][1] ^= v
        s[i][2] ^= u
        s[i][3] ^= v
    mix_columns(s)

def sub_bytes(s, sbox=s_box):
    return [[int(sbox[a]) for a in row] for row in s]

def decrypt(key, ciphertext):
    round_keys = expand_key(key) # Remember to start from the last round key and work backwards through them when decrypting

    # Convert ciphertext to state matrix
    state = bytes2matrix(ciphertext)
    
    # Initial add round key step
    state = add_round_key(state, round_keys.pop())

    for i in range(N_ROUNDS - 1, 0, -1):
        # Do round
        inv_shift_rows(state)
        state = sub_bytes(state, sbox=inv_s_box)
        state = add_round_key(state, round_keys.pop())
        inv_mix_columns(state)

    # Run final round (skips the InvMixColumns step)
    inv_shift_rows(state)
    state = sub_bytes(state, sbox=inv_s_box)
    state = add_round_key(state, round_keys.pop())

    plaintext = matrix2bytes(state) # Convert state matrix to plaintext

    return plaintext

print(decrypt(key, ciphertext))

b'crypto{MYAES128}'


### Modes of Operation Starter

In [13]:

import requests

url = "https://aes.cryptohack.org/block_cipher_starter/"
# Get the flag
flag = requests.get(url + "encrypt_flag/").json()
print(flag)
hex = requests.get(url + "decrypt/" + flag["ciphertext"] ).json().get("plaintext")
print(hex)
print(bytes.fromhex(hex).decode())

{'ciphertext': 'bf791ff22629657bedc29428674f686758cc40d847834a4665a205bf540fbdfb'}
63727970746f7b626c30636b5f633170683372355f3472335f663435375f217d
crypto{bl0ck_c1ph3r5_4r3_f457_!}


### Passwords as Keys

In [19]:
from Crypto.Cipher import AES
import requests, hashlib

def decrypt(word):
    key = hashlib.md5(word).digest()
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.decrypt(ciphertext)

wordlist = requests.get("https://gist.githubusercontent.com/wchargin/8927565/raw/d9783627c731268fb2935a731a618aa8e95cf465/words").content.splitlines()
ciphertext = bytes.fromhex(requests.get("https://aes.cryptohack.org/passwords_as_keys/encrypt_flag/").json()["ciphertext"])

for word in wordlist:
    plaintext = decrypt(word)
    try:
        print(plaintext.decode())
    except:
        continue
    

crypto{k3y5__r__n07__p455w0rdz?}


### ECB Oracle

In [25]:
import requests
import string

In [26]:
url = 'https://aes.cryptohack.org/ecb_oracle/encrypt'

In [42]:
def encrypt(cipher):
    hex_result = cipher.hex()
    r = requests.get(f"{url}/{hex_result}/")
    r.raise_for_status()
    return bytes.fromhex(r.json()['ciphertext'])

In [43]:
def get_block_size():
    encrypt_len = len(encrypt(b"A")) 
    for i in range(2, 65): 
        v = len(encrypt(b"A" * i))
        if v != encrypt_len:
            return v - encrypt_len

In [44]:
def get_flag():
    block_size = get_block_size()

    flag = b""

    for _ in range(64):
        pad_len = (block_size - 1) - (len(flag) % block_size)
        
        if pad_len == 0:
            prefix = b"A" * block_size
        else:
            prefix = b"A" * pad_len

        ct = encrypt(prefix)
        block_index = (len(prefix) + len(flag)) // block_size
        target_block = ct[block_index * block_size:(block_index + 1) * block_size]

        # brute‐force the next byte
        for c in (string.ascii_letters + string.digits + "_{}").encode():
            guess = prefix + flag + bytes([c])
            gct = encrypt(guess)
            guess_block = gct[block_index * block_size:(block_index + 1) * block_size]
            if guess_block == target_block:
                flag += bytes([c])
                print(f"Flag: {flag.decode()}")
                break

        if flag.endswith(b"}"):
            print("Flag found!")
            break

    return flag.decode()

In [38]:
flag = get_flag()
print(f"\Complete Flag: {flag}")

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
/var/folders/j4/9n23z0d56_56ygtb0hhdmhxh0000gn/T/ipykernel_35232/1522367483.py:2: SyntaxWarning: invalid escape sequence '\C'
  print(f"\Complete Flag: {flag}")


Flag: c
Flag: cr
Flag: cry
Flag: cryp
Flag: crypt
Flag: crypto
Flag: crypto{
Flag: crypto{p
Flag: crypto{p3
Flag: crypto{p3n
Flag: crypto{p3n6
Flag: crypto{p3n6u
Flag: crypto{p3n6u1
Flag: crypto{p3n6u1n
Flag: crypto{p3n6u1n5
Flag: crypto{p3n6u1n5_
Flag: crypto{p3n6u1n5_h
Flag: crypto{p3n6u1n5_h4
Flag: crypto{p3n6u1n5_h47
Flag: crypto{p3n6u1n5_h473
Flag: crypto{p3n6u1n5_h473_
Flag: crypto{p3n6u1n5_h473_3
Flag: crypto{p3n6u1n5_h473_3c
Flag: crypto{p3n6u1n5_h473_3cb
Flag: crypto{p3n6u1n5_h473_3cb}
Flag found!
\Complete Flag: crypto{p3n6u1n5_h473_3cb}


### ECB CBC WTF

In [78]:
import requests
from binascii import unhexlify

In [79]:
print(requests.get("http://aes.cryptohack.org/ecbcbcwtf/encrypt_flag").json())

{'ciphertext': '5194bfe4ef3487b45ecf9619ecd10c1de0ed114bbad84721b0957941f6a38f89afdc5db35bbc794cb3ec4f0b0d4af297'}


In [80]:
def decrypt_block(hex):
    URL_decrypt = "http://aes.cryptohack.org/ecbcbcwtf/decrypt"
    r = requests.get(f"{URL_decrypt}/{hex}/")
    plain_hex = r.json().get("plaintext")
    return unhexlify(plain_hex)

In [89]:
def get_flag():
    URL_encrypt = "http://aes.cryptohack.org/ecbcbcwtf/encrypt_flag"
    r = requests.get(URL_encrypt)
    ciphertext = r.json()["ciphertext"]

    # 16 byte block size
    iv = unhexlify(ciphertext[:32])
    blocks = [ciphertext[i:i+32] for i in range(32, len(ciphertext), 32)]
    print(f"blocks {blocks}")
    plaintext = b""
    prev = iv
    for block_hex in blocks:
        decrypted = decrypt_block(block_hex)
        pt_block = bytes(a ^ b for a, b in zip(decrypted, prev))
        plaintext += pt_block
        prev = unhexlify(block_hex)


    flag = plaintext.decode('ascii')
    print("flag:", flag)

In [88]:
get_flag()

blocks ['2da5501a4d38d7d3c3f6f6c71cf70b3c', 'f339d072908087d4baa6d4c12c7cf13c']
 flag: crypto{3cb_5uck5_4v01d_17_!!!!!}


### Flipping Cookie

In [95]:
import requests
from binascii import unhexlify, hexlify


# 1. Fetch your cookie (IV||ciphertext in hex)
r = requests.get(f"https://aes.cryptohack.org/flipping_cookie/get_cookie/")
cookie = r.json()["cookie"]
print(cookie)
iv_hex = cookie[:32]
ct_hex = cookie[32:]

# 2. Prepare deltas for positions 6..10
deltas = {
    6:  0x46 ^ 0x54,
    7:  0x61 ^ 0x72,
    8:  0x6c ^ 0x75,
    9:  0x73 ^ 0x65,
    10: 0x65 ^ 0x3b,
}

# 3. Flip bits in the IV
iv = bytearray(unhexlify(iv_hex))
for pos, d in deltas.items():
    iv[pos] ^= d
iv_mod = hexlify(iv).decode()

# 4. Send modified IV + original ciphertext
result = requests.get(f"https://aes.cryptohack.org/flipping_cookie/check_admin/{ct_hex}/{iv_mod}/")
print(result.json())


5c260840c10169a64a08e94840beb1ba763f1747188ce1925a8fb9c53e2171c258dd6a7a8717270331946cc2c203f288
{'flag': 'crypto{4u7h3n71c4710n_15_3553n714l}'}


### Symmetry

In [103]:
import requests

In [104]:
url = 'http://aes.cryptohack.org/symmetry'

In [126]:
def get_flag():
  r = requests.get(f"{url}/encrypt_flag/").json()
  ciphertext = bytes.fromhex(r['ciphertext'])

  # IV and cipher at lenght 16 based on site
  iv = ciphertext[:16]
  ciphertext = ciphertext[16:]

  response = requests.get(url=f"{url}/encrypt/{ciphertext.hex()}/{iv.hex()}").json()
  plaintext = bytes.fromhex(response['ciphertext'])
  print(plaintext)

In [127]:
print(get_flag())

b'crypto{0fb_15_5ymm37r1c4l_!!!11!}'
None
